# PTQ

Post training quantization (PTQ) is the process of reducing the precision of model weights for an already-trained model, such as pretrained model. 


In [ ]:
# Minimizing Quantization Error

# TODO
# calibration, how it works, (zero shot)


## PTQ Impact

### Faster Inference on Supported Machines

Quantization can significantly improve inference speed on hardware that supports lower-precision computations. Many modern GPUs and accelerators, such as NVIDIA's H100 GPUs, support specialized formats like `fp8`. When these formats are natively supported, the hardware can process computations directly, resulting in faster inference.

However, on hardware that does not support the target format, the quantized representation must first be converted to a supported precision (usually higher precision, `fp16`, `fp32` etc.) before computations can occur. This conversion introduces overhead, which can negate the performance gains of quantization. Therefore, quantization is most effective when used with hardware designed for lower-precision arithmetic.

> PTQ + supported hardware -> faster inference


### Faster Inference Using Specialized Kernels

To leverage the hardware computation for LLM workloads - matrix multiplications - specialized kernels can be integrated for a specific precision. Kernels are computer instructions that provide details on data manipulation and computation (ex. apply weight shuffling to align weight entries for memory access patterns for NVIDIA tensor cores to maximize throughput and minimize loading latency). One example is [Machete kernel](https://neuralmagic.com/blog/introducing-machete-a-mixed-input-gemm-kernel-optimized-for-nvidia-hopper-gpus/?utm_source=chatgpt.com), specifically optimized for H100 GPUs and model weights quantized to 4 bits. Another is Marlin, which contain instructions for A100 GPUs for `int4` weights and `fp16` activations.


To maximize the computational potential of modern hardware for LLM workloads — primarily matrix multiplications — specialized kernels are employed. Kernels are sets of computer instructions designed to optimize how data is manipulated and computations are performed. Ex. techniques like weight shuffling reorganizes weight data to align with memory access patterns, allowing NVIDIA Tensor Cores to maximize throughput and minimize loading latency. Specialized Kernels include:

* **Machete**:
Optimized for NVIDIA H100 GPUs. Designed for models with weights quantized to `int4`. By leveraging hardware features like advanced Tensor Core instructions (wgmma) and efficient memory access strategies, Machete dramatically improves inference speed and reduces memory usage - mediam time to first token to <250ms and median time per output token to <100ms with 1xH100 on Llama 3.1-70B.

* **Marlin**:
Tailored for NVIDIA A100 GPUs, Marlin supports mixed-precision computations with INT4 weights and FP16 activations. It provides optimized instructions to handle these lower-precision formats efficiently, enabling faster inference on hardware that supports these precision formats.

> PTQ + supported hardware + specific kernel -> much faster inference

Using quantization techniques, along with precison supported hardware and specialized kernels, it demonstrates how hardware and software innovations work together to handle the growing computational demands of LLMs, achieving faster inference times while using less memory.`m

# Which algo is supported on whihc kernel - here


# How do the kernels work - kernel.ipyb
# # What are the impact of using the kernel - kernel.ipyb
# In what situations do you use what - ptq_algorithms.ipyb
- in low QPS, use weight only - not much speed up in high QPS
- in high QPS or offline, use activation quantization
      A100 - w8a8 chan weights, dynamic per token act
      H100 - w8a8 fp8 chan weight, dpt